In [1]:
import matplotlib
from netCDF4 import Dataset
import datetime as dt
import json
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
import xmltodict
import yaml
import argparse
import os
import subprocess
from datetime import date
import logging as logger
import os

In [2]:
username = 'XXXXX'
password = 'XXXXX'
url = 'https://131.176.236.38/dhus/'
aoi = 'POLYGON((94.26269531249999 5.528510525692795,96.45996093749997 -50.958426723359906,179.69238281249997 -51.23440735163458,175.73730468749994 3.4256915244180703,94.26269531249999 5.528510525692795,94.26269531249999 5.528510525692795))'
awss3bucket = 's3vtaustralia/'

In [3]:
def get_file_list(username, password, aoi, startrecord):
    # rows returned is limited to 100, add pagination but looking at number of records and incrementing by 100 each iteration
    try:
        subprocess.call(['wget','--no-check-certificate', '--user='+username, '--password='+password, '--output-document=filelist.txt', 'https://131.176.236.38/dhus/search?q=footprint:"Intersects('+aoi+')" AND platformname:Sentinel-3 AND producttype:SL_2_FRP___&rows=100&start='+startrecord+'&format=json'])
        success = True
    except:
        success = False
    return(success)

In [4]:
def get_file(username, password, uuid, zipname):
    # rows returned is limited to 100, add pagination but looking at number of records and incrementing by 100 each iteration
    try:
        subprocess.call(['wget', '--no-check-certificate', '--content-disposition',  '--continue', '--user='+username, '--password='+password, "https://131.176.236.38/dhus/odata/v1/Products('"+uuid+"')/$value"])
        success = True
    except:
        success = False
    try:
        subprocess.call(['unzip', zipname])
        success = True
    except:
        success = False
    return(success)

In [2]:
def get_polygon_from_gml(gml_dict):
    listoftuples = []
    for i in list(gml_dict.split(" ")):
        pair = (float(i.split(',')[1]), float(i.split(',')[0]))
        listoftuples.append(pair)
    return(listoftuples)

In [89]:
def IPF_FRP_read(filename):
    print ("Processing ",filename)
    try:
        dataset = Dataset(filename)
    except (RuntimeError):
        print ("Unable to open ",filename)
        sys.exit(1)
    IPF_FRP =  gpd.GeoDataFrame()
    for var in dataset.variables:
        # print (var)
        temp = dataset[var]
        if len(temp.shape) < 2:
            IPF_FRP[var] = dataset[var][:]
    IPF_FRP.geometry = gpd.points_from_xy(IPF_FRP.longitude, IPF_FRP.latitude)
    return IPF_FRP    

In [7]:
# Get File List from server
# Run this if no local inventory exists
if not os.path.exists('s3vt_inventory.json'):
    startrecord = 0

    responselist = [] 

    # Determine number of records to retrieve
    get_file_list(username, password, aoi, str(startrecord))
    with open('filelist.txt') as results:
        for i in results: 
            response = json.loads(i)
            responselist.append(response)

    upperlimit = int(response['feed']['opensearch:totalResults'])
    upperlimit = 200
    # Get the full list of records

    while startrecord <= upperlimit:
        startrecord = startrecord+100

        get_file_list(username, password, aoi, str(startrecord)) 
        with open('filelist.txt') as results:
            for i in results: responselist.append(json.loads(i))

            # Dump the results to an inventory file
            with open('s3vt_inventory.json', 'w') as f:
                json.dump(responselist, f)

In [21]:
# Read inventory to geopandas - write to geojson       
        
with open('s3vt_inventory.json') as inventory:
    frames = []
    for p in inventory:
        pages = json.loads(p)
                
        for page in pages:
            try:
                for entry in page['feed']['entry']:     
                    df = pd.DataFrame.from_dict(entry, orient='index')
                    polygon = get_polygon_from_gml(xmltodict.parse(entry['str'][1]['content'])['gml:Polygon']['gml:outerBoundaryIs']['gml:LinearRing']['gml:coordinates'])

                    df = df.transpose()
                    df['Coordinates'] = Polygon(polygon)
                    for d in entry['str']:
                        if d['name'] ==  'orbitdirection':
                            df['orbitdirection'] = d['content']
                        if d['name'] ==  'platformidentifier':
                            df['platformidentifier'] = d['content'] 
                        if d['name'] ==  'filename':
                            df['filename'] = d['content']
                        if d['name'] ==  'instrumentshortname':
                            df['instrumentshortname'] = d['content']
                        if d['name'] ==  'passnumber':
                            df['passnumber'] = d['content']        
                    s3vtdf = gpd.GeoDataFrame(df, geometry='Coordinates')

                    frames.append(s3vtdf) 
            except:
                print("we have an exception")
s3vtgpd = pd.concat(frames)

# Not sure why we need to index but do it anyway
s3vtgpd = s3vtgpd.reset_index(drop=True)
s3vtgpd['date'] = pd.to_datetime(s3vtgpd.summary.str.split(",", expand= True)[0].str.split(' ', expand=True)[1])
# Some fields are lists and geojson translation doesn't like it

s3vtgpd = s3vtgpd.drop(['link', 'int', 'str', 'summary'], axis=1)
s3vtgpd.to_file('s3vt_geometry.geojson', driver='GeoJSON')

we have an exception


In [93]:
s3vtgpd

,title,id,date,Coordinates,platformidentifier,filename,instrumentshortname,orbitdirection,passnumber
0,S3A_SL_2_FRP____20200223T040332_20200223T04063...,8c5a471a-99df-41c2-8ce9-aedbed9cf5b2,2020-02-23 04:03:31.879000+00:00,"POLYGON ((79.33770 -0.07929, 79.78710 -0.17807...",2016-011A,S3A_SL_2_FRP____20200223T040332_20200223T04063...,SLSTR,descending,41847
1,S3A_SL_2_FRP____20200223T022833_20200223T02313...,ea3e05e6-23d6-4f62-8311-f55890726230,2020-02-23 02:28:32.677000+00:00,"POLYGON ((99.06370 -20.99360, 99.53060 -21.128...",2016-011A,S3A_SL_2_FRP____20200223T022833_20200223T02313...,SLSTR,descending,41845
2,S3A_SL_2_FRP____20200223T022533_20200223T02283...,5aaff91d-f33b-42db-b4f5-3cc296626d68,2020-02-23 02:25:32.677000+00:00,"POLYGON ((102.05100 -10.56830, 102.50300 -10.6...",2016-011A,S3A_SL_2_FRP____20200223T022533_20200223T02283...,SLSTR,descending,41845
3,S3A_SL_2_FRP____20200223T022233_20200223T02253...,9001c2e4-a742-4a58-a3b3-cad074ede9dd,2020-02-23 02:22:32.677000+00:00,"POLYGON ((104.58300 -0.07950, 105.02900 -0.186...",2016-011A,S3A_SL_2_FRP____20200223T022233_20200223T02253...,SLSTR,descending,41845
4,S3A_SL_2_FRP____20200223T023433_20200223T02373...,f7f289d8-5b92-40c0-a38a-4b4b6a55c769,2020-02-23 02:34:32.677000+00:00,"POLYGON ((90.59270 -41.43830, 91.15950 -41.606...",2016-011A,S3A_SL_2_FRP____20200223T023433_20200223T02373...,SLSTR,descending,41845
...,...,...,...,...,...,...,...,...,...
395,S3A_SL_2_FRP____20200219T150611_20200219T15091...,468402fd-203a-49e4-81f2-0fb248847090,2020-02-19 15:06:11.032000+00:00,"POLYGON ((110.07100 12.46630, 109.60700 12.379...",2016-011A,S3A_SL_2_FRP____20200219T150611_20200219T15091...,SLSTR,ascending,41747
396,S3A_SL_2_FRP____20200219T145512_20200219T14581...,f9425ecc-e1ca-4110-b2a3-d2ad63505aa3,2020-02-19 14:55:11.802000+00:00,"POLYGON ((119.82700 -25.84280, 119.34000 -25.9...",2016-011A,S3A_SL_2_FRP____20200219T145512_20200219T14581...,SLSTR,ascending,41745
397,S3A_SL_2_FRP____20200219T144912_20200219T14521...,d62b91b1-0681-4865-ad4b-15f424028276,2020-02-19 14:49:11.802000+00:00,"POLYGON ((129.51600 -46.07870, 128.90700 -46.2...",2016-011A,S3A_SL_2_FRP____20200219T144912_20200219T14521...,SLSTR,ascending,41745
398,S3B_SL_2_FRP____20200219T124039_20200219T12433...,d8ba2f2e-c3e7-4dfa-9095-634efcfba1cd,2020-02-19 12:40:39.221000+00:00,"POLYGON ((148.89500 -4.99734, 148.44800 -5.105...",2018-039A,S3B_SL_2_FRP____20200219T124039_20200219T12433...,SLSTR,ascending,18955


In [102]:
dataframelength = len(s3vtgpd)
# Add field to enable monitoring
s3vtgpd['hotspot'] = 0
s3vtgpd['download'] = 0
s3vtgpd['s3bucket'] = 0

s3vthostpotsgpdlist = []

In [103]:
#for i in range(dataframelength):

for i in range(4):
    if s3vtgpd.loc[i]['download'] == 0:
        zipname = s3vtgpd.loc[i]['title']+'.zip'
        uuid = s3vtgpd.loc[i]['id']
        filename = s3vtgpd.loc[i]['title']+'.SEN3/FRP_in.nc'
        s3hotspots = s3vtgpd.loc[i]['title']+'.FRP.geojson'
        print(username, password, uuid, zipname, filename)

        if get_file(username, password, uuid, zipname) == False:
            s3vtgpd.at[i, 'download'] = 0
            break
        else:
            s3vtgpd.at[i, 'download'] = 1
            s3hotspotsgpd = IPF_FRP_read(filename)
            if len(s3hotspotsgpd) != 0:
                s3hotspotsgpd.to_file(s3hotspots, driver='GeoJSON')
                s3vthostpotsgpdlist.append(s3hotspotsgpd)
                s3vtgpd.at[i, 'hotspot'] = 1
            else:
                s3vtgpd.at[i, 'hotspot'] = 0
    else:
        print(s3vtgpd.loc[i]['title']+'.zip', "already exists")

s3exp S3expert321 8c5a471a-99df-41c2-8ce9-aedbed9cf5b2 S3A_SL_2_FRP____20200223T040332_20200223T040632_20200224T094404_0179_055_161_2880_LN2_O_NT_004.zip S3A_SL_2_FRP____20200223T040332_20200223T040632_20200224T094404_0179_055_161_2880_LN2_O_NT_004.SEN3/FRP_in.nc
Processing  S3A_SL_2_FRP____20200223T040332_20200223T040632_20200224T094404_0179_055_161_2880_LN2_O_NT_004.SEN3/FRP_in.nc
s3exp S3expert321 ea3e05e6-23d6-4f62-8311-f55890726230 S3A_SL_2_FRP____20200223T022833_20200223T023133_20200224T090751_0180_055_160_3240_LN2_O_NT_004.zip S3A_SL_2_FRP____20200223T022833_20200223T023133_20200224T090751_0180_055_160_3240_LN2_O_NT_004.SEN3/FRP_in.nc
Processing  S3A_SL_2_FRP____20200223T022833_20200223T023133_20200224T090751_0180_055_160_3240_LN2_O_NT_004.SEN3/FRP_in.nc
s3exp S3expert321 5aaff91d-f33b-42db-b4f5-3cc296626d68 S3A_SL_2_FRP____20200223T022533_20200223T022833_20200224T090742_0179_055_160_3060_LN2_O_NT_004.zip S3A_SL_2_FRP____20200223T022533_20200223T022833_20200224T090742_0179_055_1

In [104]:
gpd.GeoDataFrame( pd.concat( s3vthostpotsgpdlist, ignore_index=True) )

,F1_Fire_pixel_radiance,FRP_MWIR,FRP_SWIR,FRP_uncertainty_MWIR,FRP_uncertainty_SWIR,Glint_angle,IFOV_area,Radiance_window,S7_Fire_pixel_radiance,TCWV,...,longitude,n_SWIR_fire,n_cloud,n_water,n_window,time,transmittance_MWIR,transmittance_SWIR,used_channel,geometry
0,3.88,97.685828,-1.0,10.034917,0.0,2.729426,0.0,0.60,0.75,53.800960,...,116.114962,0,0,0,5,635740046655533,0.632042,-1.0,1,POINT (116.11496 -8.28141)
1,3.78,94.696359,-1.0,9.726573,0.0,2.854888,0.0,0.60,0.75,53.184289,...,116.133810,0,0,0,5,635740046955681,0.633863,-1.0,1,POINT (116.13381 -8.28536)
2,4.24,108.328362,-1.0,11.127992,0.0,2.731208,0.0,0.60,0.75,53.640367,...,116.121923,0,0,0,5,635740046955518,0.632595,-1.0,1,POINT (116.12192 -8.28846)
3,4.24,108.287325,-1.0,11.122040,0.0,2.793524,0.0,0.60,0.76,53.300022,...,116.121923,0,0,0,5,635740046955600,0.633616,-1.0,1,POINT (116.12192 -8.28846)
4,4.34,111.271845,-1.0,11.427410,0.0,2.856431,0.0,0.60,0.76,52.959676,...,116.134932,0,0,0,5,635740046955681,0.634639,-1.0,1,POINT (116.13493 -8.29290)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1.54,28.296747,-1.0,10.635632,0.0,1.919192,1070469.0,0.71,0.75,50.166551,...,115.753539,0,5,52,9,635740047553053,0.649863,0.0,1,POINT (115.75354 -8.41184)
164,0.96,16.097757,0.0,2.797428,0.0,4.773747,0.0,0.53,0.75,51.736945,...,117.301089,-1,0,0,5,635739906557485,0.648094,0.0,1,POINT (117.30109 -0.14616)
165,0.98,78.558919,0.0,28.280853,0.0,6.073452,0.0,0.55,0.75,51.928243,...,116.378857,-1,0,0,5,635739940152320,0.653523,0.0,1,POINT (116.37886 -2.22119)
166,0.95,22.800363,0.0,6.247577,0.0,34.901598,0.0,0.55,0.75,53.807585,...,113.417485,-1,0,0,5,635739930826220,0.666602,0.0,1,POINT (113.41748 -1.56836)
